# **FUNCTIONS**  

Read the code if you want, if not just run the next cell. *i added the matchday information for sample selection, i mounted google drive to load the files from there and added few more libraries*

In [88]:
import xml.etree.ElementTree as et
import pandas as pd
from IPython.core import display as ICD
from pandas import json_normalize
import pandas as pd
import tqdm as tqdm
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from datetime import datetime
import time
from scipy.stats import mstats # for winsorize function
from sklearn.preprocessing import MinMaxScaler # MinMax normalization
import plotly.graph_objects as go # for visualizations
import plotly.express as px # for visualizations
import numpy as np


# Mount google drive

from google.colab import drive
drive.mount('/content/drive')

#@title
types_dict={1:"Pass",2:"Offside Pass",3:"Take On",4:"Foul",5:"Out",6:"Corner Awarded",7:"Tackle",8:"Interception",9:"Turnover",10:"Save",11:"Claim",12:"Clearance",13:"Miss",14:"Post",15:"Attempt Saved",16:"Goal",17:"Card",18:"Player off",19:"Player on",20:"Player retired",21:"Player returns",22:"Player becomes goalkeeper",23:"Goalkeeper becomes player",24:"Condition change",25:"Official change",27:"Start delay",28:"End delay",30:"End",31:"Picked an orange",32:"Start",34:"Team set up",35:"Player changed position",36:"Player changed Jersey number",37:"Collection End",38:"Temp_Goal",39:"Temp_Attempt",40:"Formation change",41:"Punch",42:"Good Skill",43:"Deleted event",44:"Aerial",45:"Challenge",47:"Rescinded card",49:"Ball recovery",50:"Dispossessed",51:"Error",52:"Keeper pick-up",53:"Cross not claimed",54:"Smother",55:"Offside provoked",56:"Shield ball opp",57:"Foul throw-in",58:"Penalty faced",59:"Keeper Sweeper",60:"Chance missed",61:"Ball touch",63:"Temp_Save",64:"Resume",65:"Contentious referee decision",67:"50/50",68:"Referee drop ball",69:"Failed To Block",72:"Caught offside",73:"Other Ball Contact",74:"Blocked pass"};
types = pd.DataFrame.from_dict(types_dict,orient='index').reset_index()
types.columns=["type_id","event_name"]

qualifiers_dict={1:"Long ball",2:"Cross",3:"Head pass",4:"Through ball",5:"Free kick taken",6:"Corner taken",7:"Players caught offside",8:"Goal disallowed" ,9:"Penalty",10:"Hand",11:"6-seconds violation",12:"Dangerous play",13:"Foul",14:"Last line",15:"Head",16:"Small box-centre",17:"Box-centre",18:"Out of box-centre",19:"35+ centre",20:"Right footed",21:"Other body part",22:"Regular play",23:"Fast break",24:"Set piece",25:"From corner",26:"Free kick",28:"Own goal",29:"Assisted",30:"Involved",31:"Yellow Card",32:"Second yellow",33:"Red card",34:"Referee abuse",35:"Argument",36:"Fight",37:"Time wasting",38:"Excessive celebration",39:"Crowd interaction",40:"Other reason",41:"Injury",42:"Tactical",44:"Player position",49:"Attendance figure",50:"Official position",51:"Official ID",53:"Injured player id",54:"End cause",55:"Related event ID",56:"Zone",57:"End type",59:"Jersey number",60:"Small box-right",61:"Small box-left",62:"Box-deep right",63:"Box-right",64:"Box-left",65:"Box-deep left",66:"Out of box-deep right",67:"Out of box-right",68:"Out of box-left",69:"Out of box-deep left",70:"35+ right",71:"35+ left",72:"Left footed",73:"Left",74:"High",75:"Right",76:"Low left",77:"High left",78:"Low centre",79:"High centre",80:"Low right",81:"High right",82:"Blocked",83:"Close left",84:"Close right",85:"Close high",86:"Close left and high",87:"Close right and high",88:"High claim",89:"1 on 1",90:"Deflected save",91:"Dive and deflect",92:"Catch",93:"Dive and catch",94:"Def block",95:"Back pass",96:"Corner situation",97:"Direct free",100:"Six yard blocked",101:"Saved off line",102:"Goal mouth y co-ordinate",103:"Goal mouth z co-ordinate",106:"Attacking Pass",107:"Throw-in",108:"Volley",109:"Overhead",110:"Half Volley",111:"Diving Header",112:"Scramble",113:"Strong",114:"Weak",115:"Rising",116:"Dipping",117:"Lob",118:"One Bounce",119:"Few Bounces",120:"Swerve Left",121:"Swerve Right",122:"Swerve Moving",123:"Keeper Throw",124:"Goal Kick",127:"Direction of play",128:"Punch",130:"Team formation",131:"Team player formation",132:"Dive",133:"Deflection",134:"Far Wide Left",135:"Far Wide Right",136:"Keeper Touched",137:"Keeper Saved",138:"Hit Woodwork",139:"Own Player",140:"Pass End X",141:"Pass End Y",144:"Deleted event type",145:"Formation slot",146:"Blocked x co-ordinate",147:"Blocked y co-ordinate",153:"Not past goal line",154:"Intentional assist",155:"Chipped",156:"Lay-off",157:"Launch",158:"Persistent infringement",159:"Foul and abusive language",160:"Throw-in set piece",161:"Encroachment",162:"Leaving field",163:"Entering field",164:"Spitting",165:"Professional foul",166:"Handling on the line",167:"Out of play",168:"Flick-on",169:"Leading to attempt",170:"Leading to goal",171:"Rescinded card",172:"No impact on timing",173:"Parried safe",174:"Parried danger",175:"Fingertip",176:"Caught",177:"Collected",178:"Standing",179:"Diving",180:"Stooping",181:"Reaching",182:"Hands",183:"Feet",184:"Dissent",185:"Blocked cross",186:"Scored",187:"Saved",188:"Missed",189:"Player not visible",190:"From shot off target",191:"Off the ball foul",192:"Block by hand",194:"Captain",195:"Pull Back",196:"Switch of play",197:"Team kit",198:"GK hoof",199:"Gk kick from hands",200:"Referee stop",201:"Referee delay",202:"Weather problem",203:"Crowd trouble",204:"Fire",205:"Object thrown on pitch",206:"Spectator on pitch",207:"Awaiting officials decision",208:"Referee Injury",209:"Game end",210:"Assist",211:"Overrun",212:"Length",213:"Angle",214:"Big Chance",215:"Individual Play",216:"2nd related event ID",217:"2nd assisted",218:"2nd assist",219:"Players on both posts",220:"Player on near post",221:"Player on far post",222:"No players on posts",223:"In-swinger",224:"Out-swinger",225:"Straight",226:"Suspended",227:"Resume",228:"Own shot blocked",229:"Post-match complete",230:"GK X Coordinate",231:"GK Y Coordinate",232:"Unchallenged"};
qualifiers = pd.DataFrame.from_dict(qualifiers_dict,orient='index').reset_index()
qualifiers.columns = ["qualifier_id","description"]

qualifiers_dict2 = {str(key): str(value) for key, value in qualifiers_dict.items()}


def parsef24(F24file):
  tree = et.ElementTree(file = F24file)
  games = tree.getroot()
  gameinfo = games.findall('Game')
  gameinfo = gameinfo[0]
  game_id = gameinfo.get('id')
  home_team_id = gameinfo.get('home_team_id')
  home_team_name = gameinfo.get('home_team_name')
  away_team_id = gameinfo.get('away_team_id')
  away_team_name = gameinfo.get('away_team_name')
  competition_id = gameinfo.get('competition_id')
  competition_name = gameinfo.get('competition_name')
  season_id = gameinfo.get('season_id')
  matchday = gameinfo.get('matchday') # added matchday information


  global match_events
  match_events = pd.DataFrame()
  for game in games:
      for event in game:
          evento = json_normalize(event.attrib)
          mylist = []
          for q in event:
            mylist.append(q.attrib)
          evento["qualifiers"] = ["a"]
          evento.at[0,"qualifiers"] = mylist
          match_events = pd.concat([match_events,evento])
  match_events[["id","event_id","type_id","period_id","min","sec"]] = match_events[["id","event_id","type_id","period_id","min","sec"]].astype(int)
  match_events[["y","x"]] = match_events[["y","x"]].astype(float)
  match_events = pd.merge(match_events,types, on="type_id", how = "left")
  match_events = match_events[ ['id',"event_id","type_id", "event_name" ]+ [ col for col in match_events.columns if col not in ['id',"event_id","type_id", "event_name" ] ] ]
  match_events['competition_id'] = competition_id
  match_events['game_id'] = game_id
  match_events['home_team_id'] = home_team_id
  match_events['home_team_name'] = home_team_name
  match_events['away_team_id'] = away_team_id
  match_events['away_team_name'] = away_team_name
  match_events['competition_name'] = competition_name
  match_events['season_id'] = season_id
  match_events['matchday'] = matchday # added matchday information

def explode_event(nome_df,id_evento,mytresh):
  global exploded_df
  exploded_df = pd.DataFrame()
  nome_df=nome_df[nome_df["type_id"]==id_evento]
  allqualifiers = pd.DataFrame()
  for i in tqdm.tqdm(range(len(nome_df))):
    qualifiers_detailed = json_normalize(nome_df.qualifiers.iloc[i]).T.reset_index().drop("index",axis=1).fillna("yes")
    qualifiers_detailed.columns = list(qualifiers_detailed.iloc[1])
    qualifiers_detailed = qualifiers_detailed.drop(labels=[0,1], axis=0)
    qualifiers_detailed.rename(columns=qualifiers_dict2,inplace=True)
    qualifiers_detailed["id"] = nome_df.id.iloc[i]
    allqualifiers = pd.concat([allqualifiers,qualifiers_detailed])
  exploded_df = pd.concat([exploded_df,pd.merge(nome_df.drop("qualifiers", axis=1), allqualifiers.reindex(sorted(allqualifiers.columns), axis=1).reset_index()\
                                            .drop("index", axis=1).dropna(thresh=len(allqualifiers)*mytresh, axis=1), on="id", how="outer")]).fillna("-")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**PLEASE READ**:


**types** - DataFrame with all the IDs of event types and their respective names. Print 'types.head()' to get an idea of the content.


**qualifiers** - DataFrame with all the IDs of qualifiers and their respective names.
Print 'qualifiers.head()' to get an idea of the content



---







**parsef24** - Function that transforms an Opta F24 XML feed into a pandas DataFrame named match_events. To be executed correctly, it should receive the XML file's name as a parameter (which should already be loaded into Google Colab).
Print 'match_events.head(15)' to get an idea of the content.



---




**explode_event** - Function that takes the DataFrame 'match_events' (or another one filtered by you) and the type of event you want, "exploding" all qualifiers into columns to create a new DataFrame named "exploded_df". The last parameter is the threshold that defines the minimum percentage of non-null values that the columns must contain. If it is 1, it will return only columns with no NaN values, while if it is 0, it returns all columns.


Example:
```
explode_event(match_events, 1, 0.15)
```

# **CONCATENATE XML FILES**

I saved the file in order to avoid running the process again. The file is attached with the code.

In [ ]:
# Use a dataframe to store the match events
all_matches = pd.DataFrame()

# Define the directory of the files
directory = "/content/drive/Othercomputers/My Mac/Downloads/F24 La Liga 2023"

# Set up a loop to run the parsef24 function for each xml file and concatenate all data in the all_matches dataframe
for filename in tqdm.tqdm(os.listdir(directory)):
    if filename.endswith('.xml'):
        file_path = os.path.join(directory, filename)
        parsef24(file_path)
        all_matches = pd.concat([all_matches, match_events], ignore_index=True)

# Save the file for later use

all_matches.to_csv("/content/drive/Othercomputers/My Mac/Downloads/F24 La Liga 2023/all_events_combined.csv",index=False)

  7%|▋         | 27/382 [03:33<43:11,  7.30s/it]<ipython-input-1-119bc0807ce8>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_events['competition_id'] = competition_id
<ipython-input-1-119bc0807ce8>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_events['game_id'] = game_id
<ipython-input-1-119bc0807ce8>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

# **RUN THE CELL ONLY IF YOU WANT TO LOAD THE CONCATENATED EVENTS FILE**

You can upload the concatenated file directly and proceed.


In [89]:
all_matches = pd.read_csv("/content/drive/Othercomputers/My Mac/Downloads/F24 La Liga 2023/all_events_combined.csv")

# **Sample selection**

For this task i selected the first 10 matchdays as a sample but the same code can be applied for any number of matchdays

In [90]:
# Convert matchday type to integer

all_matches['matchday'] = all_matches['matchday'].astype(int)

# Selecting the 1st 10 matchdays as a sample for our analysis

selected_matches = all_matches[all_matches['matchday']<=10]

# Check the number of selected games

selected_matches.game_id.nunique()

# Make sure we have the same number of matches for each team

# Count the home_matches for each team
home_matches = selected_matches.groupby(['home_team_id','home_team_name'])['game_id'].nunique()

# Count the home_matches for each team
away_matches = selected_matches.groupby(['away_team_id','away_team_name'])['game_id'].nunique()

# Combine the counts into a new DataFrame and calculate the total matches
matches_count = pd.concat([home_matches, away_matches], axis=1).reset_index()
matches_count.columns = ['drop','team','home_matches', 'away_matches']
matches_count['total_matches'] = matches_count['home_matches'] + matches_count['away_matches']
matches_count = matches_count.drop(columns=['drop'])

# Show the dataframe
matches_count

,team,home_matches,away_matches,total_matches
0,Athletic Club,6,4,10
1,Atlético de Madrid,5,5,10
2,Celta de Vigo,5,5,10
3,Espanyol,5,5,10
4,Barcelona,5,5,10
5,Sevilla,5,5,10
6,Mallorca,5,5,10
7,Rayo Vallecano,4,6,10
8,Real Betis,5,5,10
9,Real Madrid,4,6,10


# **Explode pass events of the selected sample**

I saved the file in order to avoid running the process again. The file is attached with the code.

In [7]:
# Explode pass events of the selected matches for further analysis

explode_event(selected_matches, 1, 0)

# Export the file for later use

exploded_df.to_csv("/content/drive/Othercomputers/My Mac/Downloads/F24 La Liga 2023/selected_games_passes.csv",index=False)

  0%|          | 0/93006 [00:00<?, ?it/s]


NotImplementedError: 

# **RUN THE CELL ONLY IF YOU WANT TO LOAD THE EXPLODED EVENTS FILE**

You can upload the exploded file directly and proceed.

In [91]:
# Load passes for analysis

exploded_df = pd.read_csv("/content/drive/Othercomputers/My Mac/Downloads/F24 La Liga 2023/selected_games_passes.csv")

# **Initial cleaning and definition of player position based on their average passing location coordinates**

*   Removed the set plays pass events
*   Added the team name for each event
*   Dropped some non relevant columns
*   Defined a position for each player based on their average passing locations

In [92]:
### Clean not needed pass events

# Remove Corners, Free kicks and Throw ins from the pass events

pass_events = exploded_df[
    ~(exploded_df[['Corner taken', 'Free kick taken', 'Throw-in']].eq('yes').any(axis=1))
]

# Add team name to each pass event

team_ids = pass_events[['home_team_id', 'home_team_name']].drop_duplicates().reset_index(drop=True)
team_ids.columns = ['team_id', 'team_name']

pass_events = pass_events.merge(team_ids, on='team_id', how='left')


### Define player position based on their average passing location coordinates

# 1. Filter pass events, group by player id and calculate the average passing locations for each player

grouped_passes = (
    pass_events.groupby('player_id', as_index=False)
    .agg({'x': 'mean', 'y': 'mean'})
)

# 2. Define position locations and apply the position function to get the position for each player

player_positions = grouped_passes.assign(
    position=grouped_passes.apply(
        lambda row: (
            'GK' if row['x'] < 13 else
            'CB' if 13 <= row['x'] <= 45 and 30 < row['y'] < 70 else
            'FB' if 13 <= row['x'] <= 53 and (row['y'] <= 30 or 70 <= row['y']) else
            'CM' if 45 < row['x'] <= 61 and 30 < row['y'] < 70 else
            'WF' if 53 < row['x'] and (row['y'] <= 30 or 70 <= row['y']) else
            'FWD' if 61 < row['x'] and 30 < row['y'] < 70 else
            'Unknown'
        ), axis=1)
)[['player_id', 'position']]

# 3, Add player positions to pass_events dataframe

pass_events = pd.merge(pass_events,player_positions,on="player_id",how="left")

### Keep only relevant columns

selected_cols = ['player_id', 'period_id', 'min', 'sec', 'team_id', 'team_name', 'outcome', 'x', 'y', 'keypass', 'assist', 'game_id', 'home_team_id',
                 'home_team_name', 'away_team_id', 'away_team_name', 'matchday', 'Angle', 'Cross',
                 'Length', 'Long ball', 'Pass End X', 'Pass End Y', 'Through ball', 'Zone', 'position']

pass_events = pass_events[selected_cols]

# replace values for easier manipulation

pass_events = pass_events.replace({'yes':1,'-':0})

# **Creation of custom contextual passing metrics**

Forward pass, Pass into penalty area, pass entering the final 3rd and progressive passes were defined for deeper analysis. Home/Away tag added for each event but i didn't use it for this analysis due to the small selected sample.

In [93]:
### Create contextual passes for deeper analysis & home/away description for each pass

# Create forward passes column (at least 5m ball progression on x axis)

pass_events['fwd_pass'] = pass_events.apply(lambda row: 1 if row['Pass End X'] >= row['x'] + 5 else 0, axis=1)

# Create passes into penalty area column

pass_events['pass_to_area'] = pass_events.apply(
    lambda row: (
        1 if (
            (row['x'] < 82 or row['y'] < 28 or row['y'] > 72) and  # pass starts outside the area
            (82 <= row['Pass End X'] and 28 <= row['Pass End Y'] <= 72)  # pass ends inside the area
        ) else 0
    ), axis=1
)

# Create columns with the passes entering the final third

pass_events['to_f3rd'] = pass_events.apply(lambda row: 1 if row['Pass End X'] >= 67 and row['x'] < 67 and row['fwd_pass'] ==1 else 0, axis=1)

# Create progressive passes columns (at least 30m, 20m or 10m progression depending on starting location of the pass)

pass_events['progressive'] = pass_events.apply(lambda row: 1 if (row['x'] < 40 and row['Pass End X'] >= row['x'] + 30) or
                                               (40 >= row['x'] < 65 and row['Pass End X'] >= row['x'] + 20) or
                                               (row['x'] >= 65 and row['Pass End X'] >= row['x'] + 10)
                                               else 0, axis=1)

# Add home or away description per event

pass_events['home_away'] = pass_events.apply(lambda row: 'home' if row['team_id'] == row['home_team_id'] else 'away', axis=1)

# Change data type for keypass and assist columns

pass_events[['keypass','assist']] = pass_events[['keypass','assist']].astype(int)

# **Success rates calculation and separation of successful passes**


*   Calculated team successful passes aggregates
*   Calculated success pct for each pass type
*   Changed the passing values from total to per game



In [94]:
# Separate successful passes only

successful_passes = pass_events[pass_events['outcome'] == 1]

# Aggregate successful passes by team

team_successful_passes = successful_passes.groupby(['team_id','team_name']).agg(
    total_passes=('team_id', 'count'),
    keypasses=('keypass', 'sum'),
    assists=('assist', 'sum'),
    crosses=('Cross', 'sum'),
    long_balls=('Long ball', 'sum'),
    through_balls=('Through ball', 'sum'),
    fwd_passes=('fwd_pass', 'sum'),
    passes_to_area=('pass_to_area', 'sum'),
    passes_to_f3rd=('to_f3rd', 'sum'),
    progressive_passes=('progressive', 'sum'),
    matchday=('matchday', 'max')
).reset_index()

# Aggregate all passes by team

team_passes = pass_events.groupby(['team_id','team_name']).agg(
    total_passes=('team_id', 'count'),  # Count of successful passes for each team
    keypasses=('keypass', 'sum'),
    assists=('assist', 'sum'),
    crosses=('Cross', 'sum'),
    long_balls=('Long ball', 'sum'),
    through_balls=('Through ball', 'sum'),
    fwd_passes=('fwd_pass', 'sum'),
    passes_to_area=('pass_to_area', 'sum'),
    passes_to_f3rd=('to_f3rd', 'sum'),
    progressive_passes=('progressive', 'sum'),
    matchday=('matchday', 'max')
).reset_index()

### Calculate the success rate across columns

# 1. Select columns to exclude from division

excluded_cols = ['team_id', 'team_name', 'keypasses', 'assists','matchday']

# 2. Select columns to divide

division_cols = team_successful_passes.columns.difference(excluded_cols)

# 3. Perform the division to calculate the success rate for each category

team_success_pct = team_successful_passes.copy()
team_success_pct[division_cols] = round(team_success_pct[division_cols].div(team_passes[division_cols]), 2)

### Calculate successful passes per game

# 1. Identify numeric columns that need to be divided (excluding 'team_id' and 'matchday')
columns_to_divide = team_successful_passes.select_dtypes(include=['number']).columns.drop(['team_id', 'matchday'])

# 2. Perform the division and round the results
for column in columns_to_divide:
    team_successful_passes[column] = (team_successful_passes[column] / team_successful_passes['matchday']).round(1)

# 3. Merge values with pct to create scores for ranking

team_passes_final = pd.merge(team_successful_passes, team_success_pct, on=('team_id','team_name'), suffixes=('', '_pct'))
team_passes_final.drop(columns=['keypasses_pct','assists_pct','matchday_pct'],inplace=True)

# **Normalization of metrics, passing scores creation and ranking**  

- Min Max normalization
- Scores calculation by taking into consideration the number of passes and their success rate in 60%-40% ratio
- Team rankings in each of these scores.

In [95]:
## Normalize metrics with minmax scaler

# List of columns to normalize

normalize_list = team_passes_final.drop(columns=['team_id', 'team_name','matchday']).columns.tolist()

# Initialize the MinMaxScaler

scaler = MinMaxScaler()
metrics = team_passes_final[normalize_list]

# Apply the scaler to the selected columns

team_passes_final[normalize_list] = scaler.fit_transform(metrics)

# Create scores by quantity and success pct

team_passes_final['total_passes_score'] = team_passes_final['total_passes'] * 0.6 + team_passes_final['total_passes_pct'] * 0.4
team_passes_final['crosses_score'] = team_passes_final['crosses'] * 0.6 + team_passes_final['crosses_pct'] * 0.4
team_passes_final['long_balls_score'] = team_passes_final['long_balls'] * 0.6 + team_passes_final['long_balls_pct'] * 0.4
team_passes_final['through_balls_score'] = team_passes_final['through_balls'] * 0.6 + team_passes_final['through_balls_pct'] * 0.4
team_passes_final['fwd_passes_score'] = team_passes_final['fwd_passes'] * 0.6 + team_passes_final['fwd_passes_pct'] * 0.4
team_passes_final['passes_to_area_score'] = team_passes_final['passes_to_area'] * 0.6 + team_passes_final['passes_to_area_pct'] * 0.4
team_passes_final['passes_to_f3rd_score'] = team_passes_final['passes_to_f3rd'] * 0.6 + team_passes_final['passes_to_f3rd_pct'] * 0.4
team_passes_final['progressive_passes_score'] = team_passes_final['progressive_passes'] * 0.6 + team_passes_final['progressive_passes_pct'] * 0.4

# select columns to rank

ranking_cols = ['total_passes_score', 'crosses_score', 'long_balls_score', 'through_balls_score', 'fwd_passes_score', 'passes_to_area_score', 'passes_to_f3rd_score', 'progressive_passes_score','keypasses','assists']

# Rank the selected columns

team_passes_ranked = team_passes_final[ranking_cols].rank(ascending=False, method='min').astype(int)

# Add the 'team_id' column back to the ranked DataFrame

team_passes_ranked['team_id'] = team_passes_final['team_id']

# Add the 'team_name' column back to the ranked DataFrame

team_passes_ranked['team_name'] = team_passes_final['team_name']

# Rearrange columns to have 'team_id' first, followed by the ranked metrics

team_passes_ranked = team_passes_ranked[['team_id', 'team_name'] + ranking_cols]

# Sum values from the third column to the last column of the DataFrame

team_passes_ranked['passing_total'] = team_passes_ranked.iloc[:, 2:].sum(axis=1)

# Rank the sums in an inverted manner (higher sums get lower rank numbers)

team_passes_ranked['passing_total'] = team_passes_ranked['passing_total'].rank(ascending=True, method='min').astype(int)

team_passes_ranked.rename(columns=lambda x: x.replace('_score', ''), inplace=True)

# Merge dataframes to store values and ranks in one dataframe

team_passes_full = pd.merge(team_successful_passes, team_passes_ranked, on=('team_id','team_name'), suffixes=('', '_rank'))

# **Visualize TEAM results with Radar Chart**

A radar chart to visualise the Team rankings. It takes one Team or two teams as an input for team comparison.

In [96]:
# Set team colors for the chart

team_colors = {
    'Athletic Club': '#BB0000',        # Red
    'Atlético de Madrid': '#FF0000',   # Bright Red
    'Celta de Vigo': '#00B2EE',        # Sky Blue
    'Espanyol': '#002EFF',             # Blue
    'Barcelona': '#004D98',            # Dark Blue
    'Sevilla': '#EE0000',              # Red
    'Mallorca': '#FFA500',             # Orange
    'Rayo Vallecano': '#DCDCDC',       # White
    'Real Betis': '#00A859',           # Green
    'Real Madrid': '#D3D3D3',          # Light gray
    'Real Sociedad': '#0055FF',        # Royal Blue
    'Valencia CF': '#FF6000',          # Orange
    'Real Valladolid': '#660099',      # Purple
    'Villarreal': '#FFD700',           # Gold
    'Osasuna': '#DD0000',              # Deep Red
    'Elche': '#007020',                # Forest Green
    'Getafe': '#0050F0',               # Cobalt Blue
    'Almería': '#D90012',              # Scarlet
    'Cádiz': '#FFE100',                # Yellow
    'Girona': '#E31936'                # Red
}


# Selection of metrics to be visualized

radar_ranks = [
    'total_passes_rank', 'fwd_passes_rank', 'passes_to_area_rank',
    'passes_to_f3rd_rank', 'progressive_passes_rank', 'through_balls_rank',
    'keypasses_rank'
]

# Store original ranks

original_ranks = team_passes_full.copy()

# Invert rankings (e.g., 1 becomes 20, 2 becomes 19, etc.) for correct display on the radar

team_stats_full_inverted = team_passes_full.copy()
for col in radar_ranks:
    team_stats_full_inverted[col] = 21 - team_stats_full_inverted[col]

# Function to prepare data for radar chart

def prepare_radar_data(team_name):
    team_data = team_stats_full_inverted[team_stats_full_inverted['team_name'] == team_name][radar_ranks + [col.replace('_rank', '') for col in radar_ranks]]
    original_team_data = original_ranks[original_ranks['team_name'] == team_name][radar_ranks]
    return team_data, original_team_data

# Function to create a radar chart for one or two teams

def create_radar_chart(team1_name, team2_name=None):
    categories = [col.replace('_rank', '').replace('_',' ').title() for col in radar_ranks]

    # Prepare data for the first team

    team1_data, original_team1_data = prepare_radar_data(team1_name)
    values1 = team1_data[radar_ranks].values.flatten().tolist()
    original_ranks1 = original_team1_data[radar_ranks].values.flatten().tolist()
    actuals1 = team1_data[[col.replace('_rank', '') for col in radar_ranks]].values.flatten().tolist()
    values1.append(values1[0])
    original_ranks1.append(original_ranks1[0])
    actuals1.append(actuals1[0])

    fig = go.Figure()

    # Add trace for the first team

    fig.add_trace(go.Scatterpolar(
        r=values1,
        theta=categories + [categories[0]],
        fill='toself',
        name=f"<b style='color:{team_colors[team1_name]};'>{team1_name}</b>",
        hoverinfo='text',
        text=[f'{cat}: {act}' for cat, orig_rank, act in zip(categories, original_ranks1, actuals1)],
        line=dict(color=team_colors[team1_name], width = 3)
    ))

    # Add trace for the second team (if provided)

    if team2_name:
        team2_data, original_team2_data = prepare_radar_data(team2_name)
        values2 = team2_data[radar_ranks].values.flatten().tolist()
        original_ranks2 = original_team2_data[radar_ranks].values.flatten().tolist()
        actuals2 = team2_data[[col.replace('_rank', '') for col in radar_ranks]].values.flatten().tolist()
        values2.append(values2[0])
        original_ranks2.append(original_ranks2[0])
        actuals2.append(actuals2[0])
        fig.add_trace(go.Scatterpolar(
            r=values2,
            theta=categories + [categories[0]],
            fill='toself',
            name=f"<b style='color:{team_colors[team2_name]};'>{team2_name}</b>",
            hoverinfo='text',
            text=[f'{cat}: {act}' for cat, orig_rank, act in zip(categories, original_ranks2, actuals2)],
            line=dict(color=team_colors[team2_name], width = 3)
        ))

    # Update layout

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                showline=False,
                range=[0, 20],
                showticklabels=False
            ),
            angularaxis=dict(
                showline=False,
                showticklabels=False,
                tickvals=[],
                ticks='',
                linewidth=0
            ),
            bgcolor='rgba(255, 182, 193, 0.3)'
        ),
        showlegend=True,
        title=dict(
            text=f'<b style="color:{team_colors[team1_name]};">{team1_name}</b>' if not team2_name else f'<b style="color:{team_colors[team1_name]};">{team1_name}</b> vs <b style="color:{team_colors[team2_name]};">{team2_name}</b>',
            x=0.5,
            y=0.99,
            xanchor='center',
            yanchor='top',
            font=dict(size=22)
        ),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.1,
            xanchor="center",
            x=0.5,
            font=dict(size=12)
        ),
        annotations=[
            dict(
                x=0.5,
                y=1.19,
                text="Passing Rankings",
                showarrow=False,
                font=dict(size=16, color='purple'),
                xanchor='center',
                yanchor='middle'
            )
        ]
    )

    # Add custom labels with manually adjustable positions

    label_positions = [
        {"x": 0.663, "y": 0.5, "text": categories[0], "angle": 90},
        {"x": 0.612, "y": 0.921, "text": categories[1], "angle": 40},
        {"x": 0.46, "y": 1.06, "text": categories[2], "angle": 355},
        {"x": 0.35, "y": 0.75, "text": categories[3], "angle": 305},
        {"x": 0.35, "y": 0.258, "text": categories[4], "angle": 65},
        {"x": 0.45, "y": -0.05, "text": categories[5], "angle": 10},
        {"x": 0.61, "y": 0.05, "text": categories[6], "angle": 320}
    ]

    for label in label_positions:
        fig.add_annotation(
            x=label["x"],
            y=label["y"],
            text=label["text"],
            showarrow=False,
            textangle=label["angle"],
            font=dict(size=13.5, color='darkblue'),
            xanchor='center',
            yanchor='middle'
        )

    # Add small squares with rank numbers for each metric for the first team

    for i, (rank, value) in enumerate(zip(original_ranks1[:-1], values1[:-1])):  # Skip the repeated first value
        fig.add_trace(go.Scatterpolar(
            r=[value],
            theta=[categories[i]],
            mode='markers+text',
            text=[rank],
            textposition='middle center',
            marker=dict(size=20, color=team_colors[team1_name], symbol='square', line=dict(color='white', width=1)),
            hoverinfo='skip',
            textfont=dict(size=12, color='white'),
            showlegend=False
        ))

    # Add small squares with rank numbers for each metric for the second team, if provided

    if team2_name:
        for i, (rank, value) in enumerate(zip(original_ranks2[:-1], values2[:-1])):  # Skip the repeated first value
            fig.add_trace(go.Scatterpolar(
                r=[value],
                theta=[categories[i]],
                mode='markers+text',
                text=[rank],
                textposition='middle center',
                marker=dict(size=20, color=team_colors[team2_name], symbol='square', line=dict(color='white', width=1)),
                hoverinfo='skip',
                textfont=dict(size=12, color='white'),
                showlegend=False
            ))

    fig.show()

# **Run the Visualization**

Select TeamA and optionally a TeamB from the team_names list to get the radar display. A very insightful visualization to identify the in-possession play style of any team in the league.

In [99]:
# Select teamA and teamB (optional) from the list

'''
team_names = [
    'Athletic Club',
    'Atlético de Madrid',
    'Celta de Vigo',
    'Espanyol',
    'Barcelona',
    'Sevilla',
    'Mallorca',
    'Rayo Vallecano',
    'Real Betis',
    'Real Madrid',
    'Real Sociedad',
    'Valencia CF',
    'Real Valladolid',
    'Villarreal',
    'Osasuna',
    'Elche',
    'Getafe',
    'Almería',
    'Cádiz',
    'Girona'
]
'''

teamA = 'Barcelona'
teamB = 'Girona' # optional, if you want to check one team only put none

create_radar_chart(teamA, teamB)

# **INDIVIDUAL PASSING METRICS**

* Calculated player passing aggregates
* Calculated player passing success rates
* Created custom metrics for player evaluation

In [100]:
### Create player aggregates and success pcts

# 1. Aggregate successful and total passes

agg_dict = {
    'player_id': 'count',
    'keypass': 'sum',
    'assist': 'sum',
    'Cross': 'sum',
    'Long ball': 'sum',
    'Through ball': 'sum',
    'fwd_pass': 'sum',
    'pass_to_area': 'sum',
    'to_f3rd': 'sum',
    'progressive': 'sum'
}

player_successful_passes = successful_passes.groupby(['player_id', 'team_name', 'position']).agg(agg_dict).rename(columns={'player_id': 'total_passes'}).reset_index()
player_passes = pass_events.groupby(['player_id', 'team_name', 'position']).agg(agg_dict).rename(columns={'player_id': 'total_passes'}).reset_index()

# 2. Calculate success rate percentages for relevant columns

division_cols = [col for col in player_successful_passes.columns if col not in ['player_id', 'team_name', 'position', 'keypasses', 'assists']]
player_success_pct = player_successful_passes.set_index(['player_id', 'team_name', 'position'])[division_cols].div(
    player_passes.set_index(['player_id', 'team_name', 'position'])[division_cols], fill_value=0
).round(2).reset_index()

# 3. Merging success percentages with the successful passes dataframe

player_passes_final = pd.merge(
    player_successful_passes,
    player_success_pct,
    on=['player_id', 'team_name', 'position'],
    suffixes=('', '_pct')
).replace(np.nan,0)

# 4. Drop unnecessary columns

player_passes_final.drop(columns=['keypass_pct', 'assist_pct'], inplace=True)

### Filter data and create custom metrics

# 1. Put a threshold for minimum passes and filter data with it

threshold = player_passes_final['total_passes'].mean() - player_passes_final['total_passes'].std()
player_passes_final = player_passes_final[player_passes_final['total_passes'] >= threshold]

# 2. Create custom metrics for player evaluation

player_passes_final['progression_index'] = (player_passes_final['progressive'] + player_passes_final['pass_to_area'] +
                                            player_passes_final['to_f3rd']) / player_passes_final['total_passes']
player_passes_final['fwd_ratio'] = player_passes_final['fwd_pass'] / player_passes_final['total_passes']
player_passes_final['creativity_index'] = (player_passes_final['Through ball'] + player_passes_final['keypass'] +
                                           player_passes_final['assist']) / player_passes_final['total_passes']
player_passes_final['long_balls_ratio'] = player_passes_final['Long ball'] / player_passes_final['total_passes']

# List of columns to use for a custom success pct for impactful passes only

pcts_to_include = [
    'Long ball_pct', 'Through ball_pct', 'fwd_pass_pct',
    'pass_to_area_pct', 'to_f3rd_pct', 'progressive_pct'
]

# Calculate the success pct for impactful passes only

player_passes_final['adjusted_success_pct'] = player_passes_final[pcts_to_include].mean(axis=1)

# 3. Rename columns for a better understanding

player_passes_final = player_passes_final.rename(columns={'keypass': 'keypasses', 'assist': 'assists', 'Cross':'crosses', 'Long ball': 'long_balls',
                                                          'Through ball':'through_balls', 'fwd_pass':'fwd_passes', 'pass_to_area':'passes_to_area',
                                                          'to_f3rd':'passes_to_f3rd', 'progressive':'progressive_passes', 'Cross_pct':'crosses_pct',
                                                          'Long ball_pct':'long_balls_pct', 'Through ball_pct':'through_balls_pct', 'fwd_pass_pct':'fwd_passes_pct',
                                                          'pass_to_area_pct':'passes_to_area_pct','to_f3rd_pct':'passes_to_f3rd_pct','progressive_pct':'progressive_passes_pct'})

# **BAR Chart function for top 10 players by position in passing metrics**

Created a bar chart function to visulize the top 10 players by position in each metric.

In [101]:
def passing_bar_chart(position, metric):

    # Filter data for the specified position

    filtered_data = player_passes_final[player_passes_final['position'] == position]

    # Check if the metric exists in the DataFrame

    if metric not in filtered_data.columns:
        print(f"Metric '{metric}' not found in DataFrame.")
        return

    # Sort by the specified metric and select the top 10 players

    top_players = filtered_data.sort_values(metric, ascending=False).head(10)

    # Set the bar colors scale

    colors = ['rgb(215,48,39)' if x <= 1/3 else 'rgb(252,141,89)' if x <= 2/3 else 'rgb(253,174,107)' for x in np.linspace(0, 1, len(top_players))]


    # Create the bar chart

    fig = go.Figure(go.Bar(
        x=top_players[metric],
        y=top_players['player_id'].astype(str),
        orientation='h',
        text=[f"{val:.1f}%" for val in top_players.get(f'{metric}_pct', [0]*len(top_players)) * 100],
        marker=dict(color=colors),
        hovertemplate=(
            "Player ID: %{y}<br>" +
            "Team: %{hovertext}<br>" +
            f"{metric.replace('_', ' ').title()}: " + "%{x}<br>" +
            "Accuracy pct: %{text}<extra></extra>"
        ),
        hovertext=top_players['team_name']
    ))

    # Configure layout

    fig.update_layout(
        title=dict(
            text=f'Top 10 {position}s by {metric.replace("_", " ").title()}',
            x=0.5,
            y=0.9,
            xanchor='center',
            yanchor='top',
            font=dict(family='Arial, bold', size=24, color='darkblue')
        ),
        xaxis_title=metric.replace('_', ' ').title(),
        yaxis=dict(
            title='Player ID',
            autorange="reversed",
            title_font=dict(size=14),
            tickfont=dict(size=12)
        ),
        plot_bgcolor='white',
    )

    fig.show()

# **RUN the BAR chart function to show player aggregated results**

select the desired player position and metric to plot

position = Select from: GK, CB, FB, CM, WF, FWD

metric = Select from: total_passes, crosses, long_balls, through_balls, fwd_passes, passes_to_area, passes_to_f3rd, progressive_passes

The bar chart is very useful to create the league top player standings in various categories of interest.

In [104]:
# Show the bar chart

position = 'WF' # Select from GK, CB, FB, CM, WF, FWD
metric = 'progressive_passes' # Select from total_passes, crosses, long_balls, through_balls, fwd_passes, passes_to_area, passes_to_f3rd, progressive_passes

passing_bar_chart(position, metric)

# **RANK players on the custom metrics**

* Winsorized at 95% to handle outliers
* Normalized values with MinMax scaler
* Ranked all players grouped by position
* Created a weighted average skill called passing impact

In [120]:
### Split the columns for the rankings

player_info = ['player_id','position','team_name']
ranks = ['progression_index','fwd_ratio','creativity_index','long_balls_ratio','adjusted_success_pct']
columns_to_include = player_info + ranks

player_rankings = player_passes_final[columns_to_include]

### Winsorize and normalize values before normalization to handle outlier values

# 1. Applying winsorizing to the technical metrics at 95%

player_rankings.loc[:, ~player_rankings.columns.isin(player_info)] = player_rankings.loc[:, ~player_rankings.columns.isin(player_info)].apply(lambda x: mstats.winsorize(x, limits=[0.05, 0.05]))

# 2. Normalize metrics with minmax scaler before ranking

# Initialize the MinMaxScaler

scaler = MinMaxScaler()
metrics = player_rankings[ranks]

# Apply the scaler to the selected columns

player_rankings[ranks] = scaler.fit_transform(metrics)

### RANK the players grouped by POSITION in 1 to 100 scale

# Create a new DataFrame to store the rankings

player_ranks = player_rankings[player_info].copy()

# Rank players in each metric

for metric in ranks:
    pct_rank = player_rankings.groupby('position')[metric].rank(pct=True, method='min', ascending=False) * 100
    pct_rank = round(pct_rank, 1)

    # Add the calculated rank as a new column to player_rankings DataFrame

    player_ranks[f'{metric}_rank'] = pct_rank

### Create an a weighted average overall skill called passing_impact

player_ranks['passing_impact'] = round(player_ranks['progression_index_rank'] * 0.25 + player_ranks['fwd_ratio_rank'] * 0.15 + player_ranks['creativity_index_rank'] * 0.30
                                       + player_ranks['long_balls_ratio_rank'] * 0.10 + player_ranks['adjusted_success_pct_rank'] * 0.20,1)

# Sort values on passing_impact

player_ranks = player_ranks.sort_values('passing_impact', ascending=False)

<ipython-input-120-bd1dca014c75>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# **Create a swarm plot to display results**

I am using a swarmplot to visulize the distribution of rankings per skill and position

In [118]:
def players_swarm(position, metric, df):

    # Filter the data for the selected position

    data_filtered = df[df['position'] == position].copy()

    # Define color clusters based on metric value

    bins = [0, 25, 50, 75, 100]
    labels = ['<25', '26-50', '51-75', '76<']
    data_filtered['color_scale'] = pd.cut(data_filtered[metric], bins=bins, labels=labels, include_lowest=True)

    # Randomly move markers on y axis to reduce overlapping

    y_base = np.ones(len(data_filtered))
    data_filtered['random_y'] = y_base + np.random.uniform(-0.3, 0.3, size=len(data_filtered))

    # Create the plot

    fig = px.scatter(data_filtered, x=metric, y='random_y',
                     color='color_scale', hover_name='player_id',
                     hover_data=[metric, 'team_name'],
                     labels={'color_scale': 'Value Cluster'})

    # Customize the hover data

    fig.update_traces(
        hovertemplate="<b>Player ID:</b> %{hovertext}<br>" +
                      "<b>Value:</b> %{x}<br>" +
                      "<b>Team:</b> %{customdata[0]}<extra></extra>",
        marker=dict(size=12, opacity=0.8)
    )

    # Customize title and axis

    fig.update_layout(
        title={
            'text': f'Swarm Plot of {metric.replace("_", " ").title()} for {position}s',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=22, color='darkblue', family='Arial, bold'),
        xaxis_title='',
        yaxis_title='',
        xaxis=dict(showgrid=False, range=[0,102]),
        yaxis=dict(showgrid=False, range=[0, 2], showticklabels=False),
        plot_bgcolor='rgba(247, 247, 247, 1)',

    # Adjust plot size

        width=900,
        height=400
    )
    fig.show()

# **Plot the Player metrics with the swarm plot**

You can choose the position and the metric from the list to display the player rankings distribution with a swarm plot. It can be a very useful tool to identify the play style of every player in the league.

In [121]:
### Choose position and metric and show the plot

position = 'FB' # Select from: GK, CB, FB, CM, WF, FWD
metric = 'progression_index_rank' # Select from: progression_index_rank, fwd_ratio_rank, creativity_index_rank, long_balls_ratio_rank, adjusted_success_pct_rank, passing impact

# Show the plot
players_swarm(position, metric, player_ranks)

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# **Analyzing the correlation between passing total rankings and actual league positions**

I explore the correlation between team rankings based on their passing performance and their final positions in the league standings. For an accurate analysis, it’s essential to consider data from the entire season rather than just a subset, such as 10 matchdays. However, examining preliminary data from a limited number of matches can still provide valuable initial insights. This analysis aims to determine if a significant relationship exists between effective passing and league success.

In [122]:
# Set the actual league positions list and add it as a column in team_ids dataframe

league_positions = [18,8,13,6,1,16,9,17,12,20,5,4,19,10,7,15,3,14,2,11]
team_ids['position'] = league_positions

# Merge the passing_total rankings with the league positions

comparison = pd.merge(team_passes_ranked[['team_name','passing_total']], team_ids[['team_name','position']], on='team_name', how='left')

# Calculate the difference in rankings as a new column

comparison['ranking_difference'] = comparison['position'] - comparison['passing_total']

# Calculate and print the correlation value

correlation = comparison['position'].corr(comparison['passing_total'])
print(f"Correlation between league positions and passing total rankings is: {correlation:.2f}")

Correlation between league positions and passing total rankings is: 0.51


# **Conclusion**

There is actually a 0.51 corellation between my passing rankings and actual league positions. While not so strong, it is significant and it could be used as a predictor. Further analysis is needed to validate the insight.